<a href="https://colab.research.google.com/github/aravindakrishnanl/educhat-recommender/blob/main/edurecommend_modelv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/aravindakrishnanl/educhat-recommender

Cloning into 'educhat-recommender'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 31 (delta 5), reused 15 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 1.81 MiB | 20.13 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
!pip install pandas numpy scikit-learn sentence-transformers faiss-cpu joblib torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 59.7 MB/s eta 0:00:00


In [ ]:

import pandas as pd, numpy as np, faiss, joblib, torch, torch.nn as nn
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

In [ ]:
users = pd.read_csv("/content/educhat-recommender/database/users.csv")
profiles = pd.read_csv("//content/educhat-recommender/database/students_profile.csv")
content = pd.read_csv("/content/educhat-recommender/database/content_items.csv")
interactions = pd.read_csv("/content/educhat-recommender/database/interactions.csv")
assess = pd.read_csv("/content/educhat-recommender/database/assessments.csv")

In [ ]:
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
content_text = (
    content["title"].fillna("") + " " +
    content["skills_tags"].fillna("").astype(str)
)
content_vecs = embed_model.encode(content_text, show_progress_bar=True)
faiss.normalize_L2(content_vecs)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
index = faiss.IndexFlatIP(content_vecs.shape[1])
index.add(content_vecs.astype("float32"))
faiss.write_index(index, "/content/artifacts_content_index.faiss")

content["embedding_vector"] = content_vecs.tolist()
content.to_pickle("/content/artifacts_content.pkl")
joblib.dump(embed_model, "/content/artifacts_sentence_encoder.pkl")

['/content/artifacts_sentence_encoder.pkl']

In [ ]:
user_perf = interactions.groupby("user_id")["correct"].mean().reset_index()
users = users.merge(user_perf, left_on="id", right_on="user_id", how="left").fillna(0)

In [ ]:
le_grade = LabelEncoder().fit(users["grade"].astype(str))
X = np.vstack([
    le_grade.transform(users["grade"].astype(str)),
    users["age"],
    users["correct"]
]).T
y = (users["correct"] > 0.5).astype(int)

ranker = LogisticRegression(max_iter=500).fit(X, y)
joblib.dump(ranker, "/content/artifacts_ranker.joblib")
joblib.dump(le_grade, "/content/artifacts_labelencoder.joblib")

['/content/artifacts_labelencoder.joblib']

In [ ]:
class TinyKnowledge(nn.Module):
    def __init__(self, in_dim=64, hidden=32):
        super().__init__()
        self.fc = nn.Sequential(nn.Linear(in_dim, hidden),
                                nn.ReLU(),
                                nn.Linear(hidden, 1),
                                nn.Sigmoid())
    def forward(self, x): return self.fc(x)

kn_model = TinyKnowledge()
torch.save(kn_model.state_dict(), "/content/artifacts_tinyknowledge.pt")

In [ ]:
!mkdir -p /content/artifacts
!mv /content/artifacts_* /content/artifacts/
!zip -r /content/trained_models.zip /content/artifacts

  adding: content/artifacts/ (stored 0%)
  adding: content/artifacts/artifacts_content_index.faiss (deflated 32%)
  adding: content/artifacts/artifacts_labelencoder.joblib (deflated 35%)
  adding: content/artifacts/artifacts_tinyknowledge.pt (deflated 22%)
  adding: content/artifacts/artifacts_content.pkl (deflated 56%)
  adding: content/artifacts/artifacts_sentence_encoder.pkl (deflated 9%)
  adding: content/artifacts/artifacts_ranker.joblib (deflated 36%)


In [ ]:
import joblib, faiss, torch
from sentence_transformers import SentenceTransformer
import numpy as np, pandas as pd

In [ ]:
embed_model = joblib.load("/content/artifacts/artifacts_sentence_encoder.pkl")
ranker = joblib.load("/content/artifacts/artifacts_ranker.joblib")
le_grade = joblib.load("/content/artifacts/artifacts_labelencoder.joblib")
content = pd.read_pickle("/content/artifacts/artifacts_content.pkl")
index = faiss.read_index("/content/artifacts/artifacts_content_index.faiss")

In [ ]:
sample_input = {
    "grade": 10,
    "age": 15,
    "performance_score": 0.72,  # average learning accuracy
    "career_goal": "IIT aspirant (Engineering Entrance)",
    "preferred_subjects": ["Mathematics", "Physics","Chemistry"],
    "learning_style": "visual",  # or "hands-on", "reading"
}


In [ ]:
query_text = f"Grade {sample_input['grade']} {sample_input['career_goal']} focusing on {', '.join(sample_input['preferred_subjects'])} using {sample_input['learning_style']} learning"
query_vec = embed_model.encode([query_text])
faiss.normalize_L2(query_vec)

In [ ]:
top_k = 20
D, I = index.search(query_vec.astype("float32"), top_k)

recommendations = content.iloc[I[0]][["title", "type", "skills_tags", "difficulty", "url"]]
recommendations.reset_index(drop=True, inplace=True)

print("🎯 Top Recommendations for the User:")
display(recommendations)

🎯 Top Recommendations for the User:


,title,type,skills_tags,difficulty,url
0,Chemistry - physical Level 2,video,['physical'],2,https://educhat.example.com/chemistry/physical_L2
1,Chemistry - physical Level 2,exercise,['physical'],2,https://educhat.example.com/chemistry/physical_L2
2,Chemistry - physical Level 2,article,['physical'],2,https://educhat.example.com/chemistry/physical_L2
3,Chemistry - physical Level 2,article,['physical'],2,https://educhat.example.com/chemistry/physical_L2
4,Chemistry - physical Level 2,article,['physical'],2,https://educhat.example.com/chemistry/physical_L2
5,Chemistry - physical Level 2,article,['physical'],2,https://educhat.example.com/chemistry/physical_L2
6,Chemistry - physical Level 2,video,['physical'],2,https://educhat.example.com/chemistry/physical_L2
7,Chemistry - physical Level 2,video,['physical'],2,https://educhat.example.com/chemistry/physical_L2
8,Chemistry - physical Level 2,article,['physical'],2,https://educhat.example.com/chemistry/physical_L2
9,Chemistry - physical Level 2,article,['physical'],2,https://educhat.example.com/chemistry/physical_L2


In [ ]:
!apt install git -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
!git config --global user.name "rahularulbalan-011105"
!git config --global user.email "23d101@psgitech.ac.in"

In [35]:
import nbformat

# Correct path to the notebook within the cloned repository
path = "/content/drive/MyDrive/rs_project/Untitled0.ipynb"

# Load notebook
with open(path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=nbformat.NO_CONVERT)

# Safely remove problematic widget metadata
if "widgets" in nb.get("metadata", {}):
    del nb["metadata"]["widgets"]

# Save back clean notebook
with open(path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)
print("✅ Cleaned notebook metadata and saved:", path)

✅ Cleaned notebook metadata and saved: /content/drive/MyDrive/rs_project/Untitled0.ipynb
